# VISTA3D - 14.3 LTS  ML - CUDA11

https://catalog.ngc.nvidia.com/orgs/nvidia/teams/monaitoolkit/models/monai_vista3d

In [0]:
%pip install -r vista3d/requirements.txt
%pip install https://github.com/erinaldidb/MONAILabel_DB/releases/download/v0.8.5/monailabel-0.8.5-py3-none-any.whl --no-deps
%pip install monai==1.4.0 pytorch-ignite --no-deps
%pip install databricks-sdk==0.36 --upgrade
%pip install mlflow[databricks]

In [0]:
dbutils.library.restartPython()

In [0]:
%run ../config/proxy_prep

In [0]:
sql_warehouse_id, table = init_widgets()
model_uc_name, serving_endpoint_name = init_model_serving_widgets()

In [0]:
init_env()

os.environ["DEST_DIR"] = "/Volumes/ema_rina/pixels_solacc/pixels_volume/monai_serving/vista3d/"

In [0]:
from vista3d.code.dbvista3dmodel import DBVISTA3DModel

model = DBVISTA3DModel()

In [0]:
from vista3d.code.dblabelapp import DBMONAILabelApp
import os

studies = "https://e2-demo-field-eng.cloud.databricks.com/"
app_dir = "./"
test = "infer"

os.putenv("MASTER_ADDR", "127.0.0.1")
os.putenv("MASTER_PORT", "1234")
        
conf = {
  "models": "segmentation",
  "preload": "false",
  "output": "dicom_seg",
  "table": table
}

app = DBMONAILabelApp(app_dir, studies, conf)

#Extract on of the already ingested images
series_uid = app.datastore().list_images()[0]
print(series_uid)
series_uid = "1.3.6.1.4.1.14519.5.2.1.9203.8273.533669585389327696272831671548"

#Override for debugging
import logging
logging.getLogger('monailabel.datastore.dicom').setLevel(logging.INFO)

In [0]:
from mlflow.models import infer_signature
from typing import Optional

input_examples = [
      { "input": { "action": "info" }},                   #retrieve informations about the monailabel server
      { "input": { "action": "activelearning/random" }},  #randomly return the next series_uid useful to label
      { "input": {                                        #train the model based on labelled series
        "train": {
          'name': 'train_01',
          'pretrained': True,
          'device': ['NVIDIA A10G'],
          'max_epochs': 50,
          'early_stop_patience': -1,
          'val_split': 0.2,
          'train_batch_size': 1,
          'val_batch_size': 1,
          'multi_gpu': True,
          'gpus': 'all',
          'dataset': 'SmartCacheDataset',
          'dataloader': 'ThreadDataLoader',
          'tracking': 'mlflow',
          'tracking_uri': '',
          'tracking_experiment_name': '',
          'model': 'segmentation'
          }
       }
      },
      { 'input': {                                        #train the model based on labelled series with mandatory fields
        'train': {
          'name': 'train_01',
          'pretrained': True,
          'max_epochs': 50,
          'val_split': 0.2,
          'train_batch_size': 1,
          'val_batch_size': 1,
          'gpus': 'all',
          'model': 'segmentation'
          }
       }
      },                      
      { 'input': {                                        #trigger the inference on a single DICOM series given the series uid, used in OHIF Viewer
        'infer': {
          'largest_cc': False,
          'device': ['NVIDIA A10G'],
          'result_extension': '.nrrd',
          'result_dtype': 'uint16',
          'result_compress': False,
          'restore_label_idx': False,
          'model': 'vista3d',
          'image': '1.2.156.14702.1.1000.16.1.2020031111365289000020001',
          'export_metrics': False,
          'export_overlays': False,
          'points': [[10,10,10],[20,20,20]], #list of x,y,z points
          'point_labels': [0,1]
          }
       }
      },
      { 'input': {                                        #trigger the inference on a single DICOM series given the series uid, used in OHIF Viewer with mandatory fields
        'infer': {
          'model': 'vista3d',
          'image': '1.2.156.14702.1.1000.16.1.2020031111365289000020001',
          'label_prompt': [1,26]
          }
       }
      },
      { 'input': {                                        #Return the file from the inference, used in OHIF Viewer
        'get_file': '/tmp/vista/bundles/vista3d/models/prediction/1.2.156.14702.1.1000.16.1.2020031111365289000020001/1.2.156.14702.1.1000.16.1.2020031111365289000020001_seg.nii.gz',
        'result_dtype': 'uint8'
       }
      },
      { 'series_uid': '1.2.156.14702.1.1000.16.1.2020031111365293700020003',
        'params' : {
          'label_prompt' : [1,26],
          'export_metrics': False,
          'export_overlays': False,
          'dest_dir': '/Volumes/ema_rina/pixels_solacc/pixels_volume/monai_serving/vista3d'
        }
      }
]

signature = infer_signature(input_examples, model_output="")
signature.inputs.to_json()

In [0]:
import pandas as pd
import json

label_prompt = ["liver", "hepatic tumor"]

label_dict_path = "vista3d/code/vista3d_bundle/data/jsons/label_dict.json"
label_dict = json.load(open(label_dict_path))
label_index = [label_dict[label.strip()] for label in label_prompt if label.strip() in label_dict]

input = { "series_uid": series_uid, "label_prompt": label_index, "export_metrics": True, "export_overlays": False, "dest_dir": "/Volumes/ema_rina/pixels_solacc/pixels_volume/monai_serving/vista3d"}

df = pd.DataFrame([input_examples[6]])

print(df.to_json())

model.load_context(context=None)
result = model.predict(None, df)

print("COMPLETED")

In [0]:
import mlflow

# Save the function as a model
with mlflow.start_run():
    mlflow.pyfunc.log_model(
        "DBVISTA3DModel",
        python_model=DBVISTA3DModel(),
        conda_env="./vista3d/conda.yaml",
        signature=signature,
        code_paths=["./vista3d", "./vista3d/code/lib", "./vista3d/code/bin"]
    )
    run_id = mlflow.active_run().info.run_id
    print(run_id)

In [0]:
model_uri = "runs:/{}/DBVISTA3DModel".format(run_id)
latest_model = mlflow.register_model(model_uri, model_uc_name)

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

scope_name = "pixels-scope"

if scope_name not in [scope.name for scope in w.secrets.list_scopes()]:
  w.secrets.create_scope(scope=scope_name)

token = w.tokens.create(comment=f'pixels_serving_endpoint_token')

w.secrets.put_secret(scope=scope_name, key="pixels_token", string_value=token.token_value)

In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")

model_version = latest_model.version

token_secret = "{{secrets/pixels-scope/pixels_token}}"

endpoint = client.create_endpoint(
    name=serving_endpoint_name,
    config={
        "served_entities": [
            {
                'name': 'pixels_monailabel-vista3d',
                'entity_name': model_uc_name,
                "entity_version": model_version,
                "workload_size": "Small",
                "workload_type": "GPU_MEDIUM",
                "scale_to_zero_enabled": True,
                'environment_vars': {
                  'DATABRICKS_TOKEN': token_secret,
                  'DATABRICKS_HOST': os.environ["DATABRICKS_HOST"],
                  'DATABRICKS_PIXELS_TABLE': os.environ["DATABRICKS_PIXELS_TABLE"],
                  'DATABRICKS_WAREHOUSE_ID': os.environ["DATABRICKS_WAREHOUSE_ID"],
                  'DEST_DIR': os.environ["DEST_DIR"]
                },
            }
        ]
    }
)

print("SERVING ENDPOINT CREATED:", serving_endpoint_name)


In [0]:
import os
import re
from PIL import Image
import matplotlib.pyplot as plt

folder_path = "/Volumes/ema_rina/pixels_solacc/pixels_volume/monai_serving/vista3d/overlays/1.3.6.1.4.1.14519.5.2.1.9203.8273.123231034424011422949554575163/1.3.6.1.4.1.14519.5.2.1.9203.8273.533669585389327696272831671548/"
image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]

def sort_key(filename):
    match = re.search(r'(\d+)_d_(\d+)\.jpg', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    return float('inf'), float('inf')

sorted_image_files = sorted(image_files, key=sort_key)

for image_file in sorted_image_files[0:50]:
    if "tumor" in image_file:
        image_path = os.path.join(folder_path, image_file)
        image = Image.open(image_path)
        plt.imshow(image)
        plt.title(image_file)
        plt.axis('off')
        plt.show()

In [0]:
import matplotlib.pyplot as plt
import SimpleITK as sitk

def display_nrrd(file_path):
    img = sitk.ReadImage(file_path)
    img_array = sitk.GetArrayFromImage(img)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(img_array[3], cmap='gray')
    plt.axis('off')
    plt.show()

display_nrrd('/tmp/tmpjv1s8eu7.nrrd')

In [0]:
%sh
cp /tmp/tmpy4t3utgg.nrrd /Volumes/ema_rina/pixels_solacc/pixels_volume/tmphb640i0m.nrrd